In [14]:
import numpy as np
import h5py
import os
import cv2
from sklearn.decomposition import PCA
from tqdm import tqdm

X = []
y = []

for file in tqdm(os.listdir("gestureData/")):
    if file.endswith(".mat"):
        user_id = int(file.split("_")[2])
        with h5py.File(f"gestureData/{file}", "r") as f:
            key = list(f.keys())[0]  # Örn. 'depth_sequence'
            sequence = f[key][:]  # shape: [num_frames, height, width]
            
            if sequence.shape[0] < 10:
                continue
            
            frames = []
            for i in range(sequence.shape[0]):
                frame = sequence[i]
                frame_np = np.array(frame)  # h5py veri yapısını NumPy'e çevir
                resized = cv2.resize(frame_np, (64, 64))
                flattened = resized.flatten()  # [4096]
                frames.append(flattened)

            sequence_matrix = np.stack(frames, axis=0)  # [num_frames, 4096]
            
            # # PCA uygulayarak sabit boyuta indir
            # pca = PCA(n_components=128)  # veya 256

            # PCA uygulayarak sabit boyuta indir
            n_samples, n_features = sequence_matrix.shape
            n_components = min(128, n_samples, n_features)  # Hata almamak için sınırla

            pca = PCA(n_components=n_components)
            sequence_pca = pca.fit_transform(sequence_matrix)  # [num_frames, n_components]
            mean_vector = sequence_pca.mean(axis=0)  # [n_components]

            # 128'e sabitle (0-padding)
            fixed_vector = np.zeros(128)
            fixed_vector[:len(mean_vector)] = mean_vector
            X.append(fixed_vector)            
            y.append(user_id)



100%|██████████| 336/336 [00:09<00:00, 36.78it/s]


In [17]:
X.shape

AttributeError: 'list' object has no attribute 'shape'

In [18]:
y.shape

AttributeError: 'list' object has no attribute 'shape'

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

X = np.array(X)
y = np.array(y) - 1  # ID'ler 1-10 arasında → 0-9 yap

y_cat = to_categorical(y)

model = Sequential([
    Dense(256, activation='relu', input_shape=(128,)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y_cat, epochs=25, batch_size=16, validation_split=0.2)


Epoch 1/25


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.1021 - loss: 2.3022 - val_accuracy: 0.0000e+00 - val_loss: 2.3231
Epoch 2/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1247 - loss: 2.2932 - val_accuracy: 0.0000e+00 - val_loss: 2.3560
Epoch 3/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0873 - loss: 2.2840 - val_accuracy: 0.0000e+00 - val_loss: 2.4086
Epoch 4/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1180 - loss: 2.2648 - val_accuracy: 0.0000e+00 - val_loss: 2.4760
Epoch 5/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1356 - loss: 2.2681 - val_accuracy: 0.0000e+00 - val_loss: 2.5441
Epoch 6/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1236 - loss: 2.2539 - val_accuracy: 0.0000e+00 - val_loss: 2.5464
Epoch 7/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1575 - loss: 2.2540 - val_accuracy: 0.0000e+00 - val_loss: 2.5611
Epoch 8/25
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1165 - loss: 2.2767 - val_accuracy: 